source of data: [Abridged life tables by province](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B44&conn_path=I2&language=en), [	Abridged life tables](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B41&conn_path=I2&language=en),<br>[Life Tables for Korea, 2023 (PDF)](https://kostat.go.kr/board.es?mid=a20108060000&bid=11746&act=view&list_no=434451)

wiki > [List of South Korean regions by life expectancy](https://en.wikipedia.org/wiki/List_of_South_Korean_regions_by_life_expectancy) / [Продолжительность жизни в регионах Республики Корея](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_регионах_Республики_Корея)

In [3]:
import pandas as pd
import math
import re

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [4]:
df_provinces = pd.read_csv('data/South_Korea_2023 (UTF8).csv', skiprows=1,
                           usecols=["By province", "By age ",
                                    "Expectation of life at age (Total) (Years)",
                                    "Expectation of life at age (Male) (Years)",
                                    "Expectation of life at age (Female) (Years)"
                                   ]
                          ) \
                 .rename(columns={"By province": "province",
                                  "By age ": "age",
                                  "Expectation of life at age (Total) (Years)": "le_total",
                                  "Expectation of life at age (Male) (Years)": "le_male",
                                  "Expectation of life at age (Female) (Years)": "le_female"
                                 }
                        )

print(df_provinces.shape)
df_provinces.head()

(374, 5)


,province,age,le_total,le_male,le_female
0,Seoul,0,85.0,82.0,88.0
1,Seoul,1,84.2,81.2,87.1
2,Seoul,5,80.2,77.2,83.2
3,Seoul,10,75.2,72.2,78.2
4,Seoul,15,70.3,67.3,73.2


In [5]:
df_country = pd.read_csv('data/South_Korea_2023-whole_country (UTF8).csv', skiprows=1,
                           usecols=["By age",
                                    "Expectation of life at age (Total) (Years)",
                                    "Expectation of life at age (Male) (Years)",
                                    "Expectation of life at age (Female) (Years)"
                                   ]
                          ) \
                 .rename(columns={"By age": "age",
                                  "Expectation of life at age (Total) (Years)": "le_total",
                                  "Expectation of life at age (Male) (Years)": "le_male",
                                  "Expectation of life at age (Female) (Years)": "le_female"
                                 }
                        )

df_country.loc[:, 'age'] = df_country.loc[:, 'age'].map(lambda st: int(st.split(' ')[0]))

df_country.insert(loc=0, column="province", value="South_Korea")

print(df_country.shape)
df_country.head()

(22, 5)


,province,age,le_total,le_male,le_female
0,South_Korea,0,83.5,80.6,86.4
1,South_Korea,1,82.7,79.8,85.6
2,South_Korea,5,78.7,75.8,81.6
3,South_Korea,10,73.8,70.9,76.7
4,South_Korea,15,68.8,65.9,71.7


<br>

In [7]:
# combine dataframes
df = pd.concat([df_country, df_provinces])

# delete redundant dataFrames
del df_country, df_provinces

# filter dataFrame
df = df.loc[df.age.isin([0, 65, 80, 100])]

df.loc[:, ['le_fΔm']] = df['le_female'] - df['le_male']

print(df.shape)
df.head()

(72, 6)


,province,age,le_total,le_male,le_female,le_fΔm
0,South_Korea,0,83.5,80.6,86.4,5.8
14,South_Korea,65,21.5,19.2,23.6,4.4
17,South_Korea,80,9.7,8.3,10.7,2.4
21,South_Korea,100,2.2,1.8,2.3,0.5
0,Seoul,0,85.0,82.0,88.0,6.0


In [8]:
# transform dataFrame
df = df.pivot(index='province', columns='age')

df.index.name = ''

print(df.shape)
df.head()

(18, 16)


le_total                 le_male                 le_female  \
age                    0     65   80   100     0     65   80   100       0     
                                                                               
Busan                 82.6  20.8  9.1  1.9    79.7  18.7  8.0  1.7      85.6   
Chungcheongbuk-do     82.4  20.8  9.2  2.0    79.4  18.7  7.9  1.7      85.4   
Chungcheongnam-do     83.0  21.3  9.7  2.3    80.0  19.0  8.2  1.8      86.1   
Daegu                 83.2  21.3  9.4  2.1    80.2  19.1  8.1  1.8      86.2   
Daejeon               83.6  21.5  9.8  2.3    81.1  19.5  8.5  1.9      86.1   

                                   le_fΔm                 
age                 65    80   100    0    65   80   100  
                                                          
Busan              22.8  10.0  2.0    5.9  4.1  2.0  0.3  
Chungcheongbuk-do  22.8  10.2  2.1    6.0  4.1  2.3  0.4  
Chungcheongnam-do  23.5  10.9  2.4    6.1  4.5  2.7  0.6  
Daegu              23.3  10.5  2.2    6.0  4.2  2.4  0.4  
Daejeon            23.4  10.8  2.4    5.0  3.9  2.3  0.5

In [9]:
# rename columns
rename_dict = {
    ('le_total', 0)   : 't_0',
    ('le_total', 65)  : 't_65',
    ('le_total', 80)  : 't_80',
    ('le_total', 100) : 't_100',
    ('le_male', 0)   : 'm_0',
    ('le_male', 65)  : 'm_65',
    ('le_male', 80)  : 'm_80',
    ('le_male', 100) : 'm_100',
    ('le_female', 0)   : 'f_0',
    ('le_female', 65)  : 'f_65',
    ('le_female', 80)  : 'f_80',
    ('le_female', 100) : 'f_100',
    ('le_fΔm', 0)   : 'd_0',
    ('le_fΔm', 65)  : 'd_65',
    ('le_fΔm', 80)  : 'd_80',
    ('le_fΔm', 100) : 'd_100'
}

df.columns = [rename_dict[col] for col in df.columns]

df = df.sort_values(by=['t_0', 't_65'], ascending=False)
df = pd.concat([df.loc[["South_Korea"]], df.drop(index="South_Korea")])

df.head()

,t_0,t_65,t_80,t_100,m_0,m_65,m_80,m_100,f_0,f_65,f_80,f_100,d_0,d_65,d_80,d_100
,,,,,,,,,,,,,,,,
South_Korea,83.5,21.5,9.7,2.2,80.6,19.2,8.3,1.8,86.4,23.6,10.7,2.3,5.8,4.4,2.4,0.5
Seoul,85.0,22.6,10.5,2.7,82.0,20.3,9.0,2.1,88.0,24.9,11.8,2.8,6.0,4.6,2.8,0.7
Gyeonggi-do,83.9,21.6,9.8,2.3,81.2,19.6,8.5,2.0,86.6,23.6,10.8,2.3,5.4,4.0,2.3,0.3
Jeju-do,83.7,22.2,10.6,3.0,79.7,19.1,8.2,1.8,87.7,25.2,12.4,3.1,8.0,6.1,4.2,1.3
Sejong,83.7,20.9,8.9,1.9,81.3,18.7,7.4,1.4,86.2,23.1,10.1,1.9,4.9,4.4,2.7,0.5


<br>

In [11]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
mal.min_and_max_values(df, row_center="South_Korea", nmb=3, max_lng=11)

Number of records: 18


,t_0,t_65,t_80,t_100,m_0,m_65,m_80,m_100,f_0,f_65,f_80,f_100,d_0,d_65,d_80,d_100
max,85.0 -Seoul,22.6 -Seoul,10.6 -Jeju-do,3.0 -Jeju-do,82.0 -Seoul,20.3 -Seoul,9.0 -Seoul,2.1 -Seoul,88.0 -Seoul,25.2 -Jeju-do,12.4 -Jeju-do,3.1 -Jeju-do,8.0 -Jeju-do,6.1 -Jeju-do,4.2 -Jeju-do,1.3 -Jeju-do
max_2,83.9 -Gyeonggi-do,22.2 -Jeju-do,10.5 -Seoul,2.7 -Seoul,81.3 -Sejong,19.6 -Gyeonggi-do,8.5 -Gyeonggi-do,2.0 -Gyeonggi-do,87.7 -Jeju-do,24.9 -Seoul,11.8 -Seoul,2.8 -Seoul,6.7 -Gangwon-do,4.7 -Gyeongsang…,2.9 -Gangwon-do,0.7 -Seoul
max_3,83.7 -Jeju-do,21.6 -Gyeonggi-do,9.8 -Gyeonggi-do,2.3 -Gyeonggi-do,81.2 -Gyeonggi-do,19.5 -Daejeon,8.5 -Daejeon,1.9 -Daejeon,86.6 -Gyeonggi-do,23.6 -South_Korea,10.9 -Chungcheon…,2.4 -Daejeon,6.5 -Jeollanam-…,4.7 -Jeollanam-…,2.8 -Seoul,0.7 -Jeollanam-…
South_Korea,– 83.5 –,– 21.5 –,– 9.7 –,– 2.2 –,– 80.6 –,– 19.2 –,– 8.3 –,– 1.8 –,– 86.4 –,– 23.6 –,– 10.7 –,– 2.3 –,– 5.8 –,– 4.4 –,– 2.4 –,– 0.5 –
min_3,82.6 -Busan,20.8 -Chungcheon…,9.1 -Busan,1.9 -Busan,79.5 -Gangwon-do,18.7 -Sejong,7.8 -Jeollanam-…,1.6 -Jeollanam-…,85.6 -Busan,22.8 -Chungcheon…,10.1 -Sejong,2.0 -Busan,5.1 -Ulsan,4.0 -Gyeonggi-do,2.0 -Busan,0.3 -Busan
min_2,82.5 -Jeollanam-…,20.8 -Busan,9.0 -Ulsan,1.9 -Ulsan,79.4 -Chungcheon…,18.6 -Jeollanam-…,7.7 -Gyeongsang…,1.6 -Gyeongsang…,85.5 -Ulsan,22.8 -Busan,10.0 -Busan,1.9 -Ulsan,5.0 -Daejeon,3.9 -Daejeon,2.0 -Gwangju,0.3 -Gyeonggi-do
min,82.4 -Chungcheon…,20.8 -Gyeongsang…,8.9 -Sejong,1.9 -Sejong,79.3 -Jeollanam-…,18.4 -Gyeongsang…,7.4 -Sejong,1.4 -Sejong,85.4 -Chungcheon…,22.7 -Ulsan,9.8 -Ulsan,1.9 -Sejong,4.9 -Sejong,3.6 -Ulsan,1.8 -Ulsan,0.1 -Ulsan


<br>

In [13]:
# calculate and insert "bonus" columns
df.insert(loc=1, column='bonus_0→65', value=(df['t_65']+65-df['t_0']).round(3))
df.insert(loc=3, column='bonus_65→80', value=(df['t_80']+15-df['t_65']).round(3))
df.insert(loc=5, column='bonus_80→100', value=(df['t_100']+20-df['t_80']).round(3))

df.head()

,t_0,bonus_0→65,t_65,bonus_65→80,t_80,bonus_80→100,t_100,m_0,m_65,m_80,m_100,f_0,f_65,f_80,f_100,d_0,d_65,d_80,d_100
,,,,,,,,,,,,,,,,,,,
South_Korea,83.5,3.0,21.5,3.2,9.7,12.5,2.2,80.6,19.2,8.3,1.8,86.4,23.6,10.7,2.3,5.8,4.4,2.4,0.5
Seoul,85.0,2.6,22.6,2.9,10.5,12.2,2.7,82.0,20.3,9.0,2.1,88.0,24.9,11.8,2.8,6.0,4.6,2.8,0.7
Gyeonggi-do,83.9,2.7,21.6,3.2,9.8,12.5,2.3,81.2,19.6,8.5,2.0,86.6,23.6,10.8,2.3,5.4,4.0,2.3,0.3
Jeju-do,83.7,3.5,22.2,3.4,10.6,12.4,3.0,79.7,19.1,8.2,1.8,87.7,25.2,12.4,3.1,8.0,6.1,4.2,1.3
Sejong,83.7,2.2,20.9,3.0,8.9,13.0,1.9,81.3,18.7,7.4,1.4,86.2,23.1,10.1,1.9,4.9,4.4,2.7,0.5


<br>

In [15]:
# for region in df.sort_index().index.to_list():
#     print(f"    '{region}'\t: {{'ko': ('', ''), 'en': ('', ''), 'ru': ('', '')}},")

In [16]:
dd_replacement = {
    'South_Korea'	: {'ko': ('대한민국', ''), 'en': ('South Korea', ''), 'ru': ('Южная Корея', '')},
    'Busan'	: {'ko': ('부산광역시', ''), 'en': ('Busan', 'Busan'), 'ru': ('Пуса́н', 'Пусан')},
    'Chungcheongbuk-do'	: {'ko': ('충청북도', ''), 'en': ('North Chungcheong Province', 'North Chungcheong Province'), 'ru': ('Чхунчхо́н-Пукто́', 'Чхунчхон-Пукто')},
    'Chungcheongnam-do'	: {'ko': ('충청남도', ''), 'en': ('South Chungcheong Province', 'South Chungcheong Province'), 'ru': ('Чхунчхо́н-Намдо́', 'Чхунчхон-Намдо')},
    'Daegu'	: {'ko': ('대구광역시', ''), 'en': ('Daegu', 'Daegu'), 'ru': ('Тэгу́', 'Тэгу')},
    'Daejeon'	: {'ko': ('대전광역시', ''), 'en': ('Daejeon', 'Daejeon'), 'ru': ('Тэджо́н', 'Тэджон')},
    'Gangwon-do'	: {'ko': ('강원특별자치도', ''), 'en': ('Gangwon Province', 'Gangwon Province, South Korea'), 'ru': ('Канвондо́', 'Канвондо (Республика Корея)')},
    'Gwangju'	: {'ko': ('광주광역시', ''), 'en': ('Gwangju', 'Gwangju'), 'ru': ('Кванджу́', 'Кванджу')},
    'Gyeonggi-do'	: {'ko': ('경기도', ''), 'en': ('Gyeonggi Province', 'Gyeonggi Province'), 'ru': ('Кёнгидо́', 'Кёнгидо')},
    'Gyeongsangbuk-do'	: {'ko': ('경상북도', ''), 'en': ('North Gyeongsang Province', 'North Gyeongsang Province'), 'ru': ('Кёнса́н-Пукто́', 'Кёнсан-Пукто')},
    'Gyeongsangnam-do'	: {'ko': ('경상남도', ''), 'en': ('South Gyeongsang Province', 'South Gyeongsang Province'), 'ru': ('Кёнса́н-Намдо́', 'Кёнсан-Намдо')},
    'Incheon'	: {'ko': ('인천광역시', ''), 'en': ('Incheon', 'Incheon'), 'ru': ('Инчхо́н', 'Инчхон')},
    'Jeju-do'	: {'ko': ('제주특별자치도', ''), 'en': ('Jeju Province', 'Jeju Province'), 'ru': ('Чеджудо́', 'Чеджудо')},
    'Jeollabuk-do'	: {'ko': ('전북특별자치도', ''), 'en': ('North Jeolla Province', 'North Jeolla Province'), 'ru': ('Чолла-Пукто', 'Чолла-Пукто')},
    'Jeollanam-do'	: {'ko': ('전라남도', ''), 'en': ('South Jeolla Province', 'South Jeolla Province'), 'ru': ('Чолла-Намдо', 'Чолла-Намдо')},
    'Sejong'	: {'ko': ('세종특별자치시', ''), 'en': ('Sejong', 'Sejong City'), 'ru': ('Седжон', 'Седжон (город)')},
    'Seoul'	: {'ko': ('서울특별시', ''), 'en': ('Seoul', 'Seoul'), 'ru': ('Сеу́л', 'Сеул')},
    'Ulsan'	: {'ko': ('울산광역시', ''), 'en': ('Ulsan', 'Ulsan'), 'ru': ('Ульса́н', 'Ульсан')},
}

In [17]:
# create code for Wikipedia table
def create_table_code_v1(df, lang='en'):

    def if_value(x, prec=1):
        # decoration of regular values
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=1):  # 'change_value'
        # decoration of colored values, that shows change of regular values
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f'color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f'color:darkgray;"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=1):
        # decoration of colored values of bold font
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    # Load code for header of the table from file. Depending on condions (see above), code is loaded from various files.
    with open(f"design/South_Korea_header_2023_v1_{lang}.txt", mode='r', encoding="utf-8") as fh:
        st_header = fh.read().strip()
    
    # Create table code record by record
    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        
        # records in bold
        if ser.name == "South_Korea":
            st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{dd_replacement[ser.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_0"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_0→65"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_65"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_65→80"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_80"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_80→100"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_100"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| \'\'\'{if_value(ser["m_0"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_65"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_80"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_100"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;border-left-width:2px;"| \'\'\'{if_value(ser["f_0"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_65"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_80"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_100"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["d_0"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_65"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_80"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_100"])}\'\'\''
        else:
            name_link = dd_replacement[ser.name][lang][1]
            name_visible = dd_replacement[ser.name][lang][0]
            name_inserted = name_link if name_link == name_visible else f"{name_link}|{name_visible}"
            st += '\n' + '|-\n' + \
                  f'| [[{name_inserted}]] ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_0"])} ' + \
                  f'|| {if_value(ser["bonus_0→65"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_65"])} ' + \
                  f'|| {if_value(ser["bonus_65→80"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_80"])} ' + \
                  f'|| {if_value(ser["bonus_80→100"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_100"])} ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| {if_value(ser["m_0"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_65"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_80"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_100"])} ' + \
                  f'||style="background:#fee7f6;border-left-width:2px;"| {if_value(ser["f_0"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_65"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_80"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_100"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["d_0"])} ' + \
                  f'|| {if_value(ser["d_65"])} ' + \
                  f'|| {if_value(ser["d_80"])} ' + \
                  f'|| {if_value(ser["d_100"])}'

    st += '\n|}'
    
    # fix microbugs that can happened due to nuances of calculations and rounding
    st = st.replace('color:darkgray;"| 0.0 ', 'color:darkgray;"| 0.0 ')  \
           .replace('color:crimson;"| −0.0 ', 'color:darkgray;"| 0.0 ')  \
           .replace('color:darkgreen;"| 0.0 ', 'color:darkgray;"| 0.0 ')  \
           .replace('color:darkgray;"| \'\'\'0.0\'\'\' ', 'color:darkgray;"| \'\'\'0.0\'\'\' ')  \
           .replace('color:crimson;"| \'\'\'−0.0\'\'\' ', 'color:darkgray;"| \'\'\'0.0\'\'\' ')  \
           .replace('color:darkgreen;"| \'\'\'0.0\'\'\' ', 'color:darkgray;"| \'\'\'0.0\'\'\' ')
      
    # since in Russian standard decimal numbers use a comma, made in numbers replacement of dot to comma
    # WORKING WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        
    # set gray color of text for missing values that in the table are represented with '—'
    st = st.replace(';"| —', ';color:silver;"| —')
    
    # join code of header of the table with code for records
    st = st_header + st
    
    return st

table_code = create_table_code_v1(df, lang='en')
with open('output/Table code for South Korean regions -v1 -en.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)
    
# table_code = create_table_code_v1(df, lang='ru')
# with open('output/Table code for South Korean regions -v1 -ru.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

In [18]:
# create code for Wikipedia table
def create_table_code_v2(df, lang='en'):

    def if_value(x, prec=1):
        # decoration of regular values
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=1):  # 'change_value'
        # decoration of colored values, that shows change of regular values
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f'color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f'color:darkgray;"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=1):
        # decoration of colored values of bold font
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    # Load code for header of the table from file. Depending on condions (see above), code is loaded from various files.
    with open(f"design/South_Korea_header_2023_v2_{lang}.txt", mode='r', encoding="utf-8") as fh:
        st_header = fh.read().strip()
    
    # Create table code record by record
    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        
        # records in bold
        if ser.name == "South_Korea":
            st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{dd_replacement[ser.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_0"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_0"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_0"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_0"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_0→65"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_65"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_65"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_65"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_65"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_65→80"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_80"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_80"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_80"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_80"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_80→100"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_100"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_100"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_100"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["d_100"])}\'\'\''
        else:
            name_link = dd_replacement[ser.name][lang][1]
            name_visible = dd_replacement[ser.name][lang][0]
            name_inserted = name_link if name_link == name_visible else f"{name_link}|{name_visible}"
            st += '\n' + '|-\n' + \
                  f'| [[{name_inserted}]] ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_0"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_0"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_0"])} ' + \
                  f'|| {if_value(ser["d_0"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_0→65"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_65"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_65"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_65"])} ' + \
                  f'|| {if_value(ser["d_65"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_65→80"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_80"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_80"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_80"])} ' + \
                  f'|| {if_value(ser["d_80"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_80→100"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_100"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_100"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_100"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["d_100"])}'

    st += '\n|}'
    
    # fix microbugs that can happened due to nuances of calculations and rounding
    st = st.replace('color:darkgray;"| 0.0 ', 'color:darkgray;"| 0.0 ')  \
           .replace('color:crimson;"| −0.0 ', 'color:darkgray;"| 0.0 ')  \
           .replace('color:darkgreen;"| 0.0 ', 'color:darkgray;"| 0.0 ')  \
           .replace('color:darkgray;"| \'\'\'0.0\'\'\' ', 'color:darkgray;"| \'\'\'0.0\'\'\' ')  \
           .replace('color:crimson;"| \'\'\'−0.0\'\'\' ', 'color:darkgray;"| \'\'\'0.0\'\'\' ')  \
           .replace('color:darkgreen;"| \'\'\'0.0\'\'\' ', 'color:darkgray;"| \'\'\'0.0\'\'\' ')
      
    # since in Russian standard decimal numbers use a comma, made in numbers replacement of dot to comma
    # WORKING WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        
    # set gray color of text for missing values that in the table are represented with '—'
    st = st.replace(';"| —', ';color:silver;"| —')
    
    # join code of header of the table with code for records
    st = st_header + st
    
    return st

table_code = create_table_code_v2(df, lang='ru')
with open('output/Table code for South Korean regions -v2 -ru.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br>
<br>

manual check

In [20]:
# values for South Korea
t_0 = 83.5
t_65 = 21.5
t_80 = 9.7
t_100 = 2.2

m_0 = 80.6
m_65 = 19.2
m_80 = 8.3
m_100 = 1.8

f_0 = 86.4
f_65 = 23.6
f_80 = 10.7
f_100 = 2.3

In [21]:
# values for Seoul
t_0 = 85.0
t_65 = 22.6
t_80 = 10.5
t_100 = 2.7

m_0 = 82.0
m_65 = 20.3
m_80 = 9.0
m_100 = 2.1

f_0 = 88.0
f_65 = 24.9
f_80 = 11.8
f_100 = 2.8

In [22]:
# sex gap
print(f"  0: {f_0  - m_0  :.1f}")
print(f" 65: {f_65 - m_65 :.1f}")
print(f" 80: {f_80 - m_80 :.1f}")
print(f"100: {f_100- m_100:.1f}")

  0: 6.0
 65: 4.6
 80: 2.8
100: 0.7


In [23]:
# bonus
print(f" 0→65 : {t_65 + 65 - t_0 :4.1f}")
print(f"65→80 : {t_80 + 15 - t_65:4.1f}")
print(f"80→100: {t_100+ 20 - t_80:4.1f}")

 0→65 :  2.6
65→80 :  2.9
80→100: 12.2
